# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

In [53]:
import requests
from bs4 import BeautifulSoup
import time
import re
import pandas as pd
import json

## Задание 1

In [54]:
URL = 'https://habr.com/ru/search/'
KEYWORDS = ['python', 'парсинг']

In [55]:
def get_all_links(URL, KEYWORDS):
    
    all_links = []
    
    # single keyword
    keyword = ('+').join(KEYWORDS)
    
    params = {
        'q': keyword,
        'target_type' : 'posts',
        'order_by' : 'relevance'
    }
    
    # first page
    res = requests.get(URL, params)
    time.sleep(0.3)
    soup = BeautifulSoup(res.text, 'html.parser')
    post_title = soup.find_all('a', class_='post__title_link')
    all_links += list(map(lambda x: x.get('href'), post_title))
    
    # number of pages
    last_page = soup.find_all('a', class_='toggle-menu__item-link toggle-menu__item-link_pagination toggle-menu__item-link_bordered')
    last_link = last_page[0].get('href')
    total_pages = re.findall(r'\d\d', last_link)
    total_pages = int(total_pages[0]) + 1
    
    # all other pages
    for i in range(2, total_pages):
        res = requests.get(URL + 'page' + str(i) + '/', params)
        time.sleep(0.3)
        soup = BeautifulSoup(res.text, 'html.parser')
        post_title = soup.find_all('a', class_='post__title_link')
        all_links += list(map(lambda x: x.get('href'), post_title))
        
    return all_links

all_links = get_all_links(URL, KEYWORDS)
all_links[:10]

['https://habr.com/ru/post/127584/',
 'https://habr.com/ru/post/504900/',
 'https://habr.com/ru/post/239081/',
 'https://habr.com/ru/post/450834/',
 'https://habr.com/ru/company/mailru/blog/420679/',
 'https://habr.com/ru/post/519796/',
 'https://habr.com/ru/post/500498/',
 'https://habr.com/ru/post/273253/',
 'https://habr.com/ru/post/446488/',
 'https://habr.com/ru/post/317304/']

In [56]:
def kw_search(all_links):
    kw_search = pd.DataFrame()
    limit = 15
    for i, link in enumerate(all_links):
        soup = BeautifulSoup(requests.get(link).text, 'html.parser')
        time.sleep(0.3)
        date = soup.find('span', class_='post__time')
        date = date.get('data-time_published')
        title = soup.find('h1', class_='post__title').text
        title = title.strip()
        text = soup.find('div', class_='post__text').text
        text = text.strip()
        row = {'date': date, 'title': title, 'text': text}
        kw_search = pd.concat([kw_search, pd.DataFrame([row])])
        kw_search.reset_index(drop=True, inplace=True)
        if i == limit:
            break
    return kw_search

In [57]:
search_res = kw_search(all_links)
search_res.head()

,date,title,text
0,2011-09-02T10:37Z,Grab — python библиотека для парсинга сайтов,"Лет пять-шесть назад, когда я ещё программиров..."
1,2020-07-23T12:17Z,Как проанализировать рынок фотостудий с помощь...,В интернете огромное количество открытых данны...
2,2014-10-03T08:03Z,Парсим на Python: Pyparsing для новичков,Парсинг (синтаксический анализ) представляет с...
3,2019-05-07T11:51Z,Парсинг сайтов — а это вообще легально в России?,По одному из определений парсинг есть синтакси...
4,2018-08-24T07:22Z,Новые курсы по Python от Mail.Ru Group,"Python — простой, гибкий и популярный язык, пр..."


## Задание 2

https://www.avast.com/hackcheck/

In [31]:
emails = ['xxx@gmail.com', 'yyy@gmail.com']

params = {
    'emailAddresses': emails
}
headers = {
    'Vaar-Version': '0', 
    'Vaar-Header-App-Product' : 'hackcheck-web-avast'
}

url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data?'

r = requests.post(url, data = json.dumps(params), headers = headers)
# r.json()

df = pd.DataFrame(r.json()['breaches'])
df = df.transpose().reset_index()
df = df.rename(columns={'index': 'breach'})

df.head()

,breach,site,recordsCount,description,publishDate
0,16794,socialengineered.net,90186,"In June 2019, hacking forum SocialEngineered's...",2019-06-27T00:00:00Z
1,3883,brainybatch.com,872730,"In January 2017, BrainyBatch Technologies' use...",2017-04-20T00:00:00Z
2,16962,cprewritten.net,1687049,"In July 2019, virtual world Club Penguin Rewri...",2019-09-26T00:00:00Z
3,16372,bookmate.com,7775238,"In July 2018, Bookmate was allegedly breached....",2019-03-20T00:00:00Z
4,16591,roll20.net,4006528,"In January 2019, Roll20 allegedly breached. Th...",2019-03-20T00:00:00Z
